### Consigna 

1. **Inversión de Matrices:**
   Completar la función `invertir_matriz(A)` para calcular la **inversa** de la matriz \( A \) utilizando **eliminación gaussiana** y sustitución hacia atrás. 

2. **Sistema de Ecuaciones Lineales:**
   Completar la función `resolver_sistema(A, b)` para resolver el sistema lineal \( A \mathbf{x} = \mathbf{b} \). Utilizar la función `invertir_matriz(A)`.

3. **Cambio de Base:**
   Completar la función `cambio_base_matriz(B_prime, B)` para calcular la matriz de **cambio de base** de \( B \) a \( B' \).


In [22]:
import numpy as np

def escalonar_filas(M):
    """ 
        Retorna la Matriz Escalonada por Filas 
    """
    A = np.copy(M)
    if (issubclass(A.dtype.type, np.integer)):
        A = A.astype(float)

    # Si A no tiene filas o columnas, ya esta escalonada
    f, c = A.shape
    if f == 0 or c == 0:
        return A

    # buscamos primer elemento no nulo de la primera columna
    i = 0
    
    while i < f and A[i,0] == 0:
        i += 1

    if i == f:
        # si todos los elementos de la primera columna son ceros
        # escalonamos filas desde la segunda columna
        B = escalonar_filas(A[:,1:])
        
        # y volvemos a agregar la primera columna de zeros
        return np.block([A[:,:1], B])


    # intercambiamos filas i <-> 0, pues el primer cero aparece en la fila i
    if i > 0:
        A[[0,i],:] = A[[i,0],:]

    # PASO DE TRIANGULACION GAUSSIANA:
    # a las filas subsiguientes les restamos un multiplo de la primera
    A[1:,:] -= (A[0,:] / A[0,0]) * A[1:,0:1]

    # escalonamos desde la segunda fila y segunda columna en adelante
    B = escalonar_filas(A[1:,1:])

    # reconstruimos la matriz por bloques adosando a B la primera fila 
    # y la primera columna (de ceros)
    return np.block([ [A[:1,:]], [ A[1:,:1], B] ])


def back_substitution(A_aug):
    """
    Realiza la sustitución hacia atrás para obtener la identidad en el lado izquierdo
    de la matriz aumentada y la inversa en el lado derecho.
    """
    n = A_aug.shape[0]
    
    # Desde la última fila hacia la primera
    for i in range(n-1, -1, -1):
        # Normalizar la fila de pivote
        A_aug[i] = A_aug[i] / A_aug[i, i]
        
        # Hacer ceros en las filas superiores
        for j in range(i):
            A_aug[j] -= A_aug[i] * A_aug[j, i]
    
    return A_aug[:, n:]


In [28]:
A = np.array([[5,  6,  0],
       [0,  1,  11],
       [0,  58,  1]])

n = A.shape[0]
I = np.identity(n)
conc = np.concatenate((A, I), axis = 1)

M = escalonar_filas(conc)

np.round(back_substitution(M), 2)

array([[ 0.2 ,  0.  , -0.02],
       [ 0.  , -0.  ,  0.02],
       [-0.  ,  0.09, -0.  ]])

In [52]:
def invertir_matriz(A):
    n = A.shape[0]
    I = np.identity(n)
    conc = np.concatenate((A, I), axis = 1)

    M = escalonar_filas(conc)

    A_inv = np.round(back_substitution(M), 2)


    return A_inv


def resolver_sistema(A, b):
    A_inv = invertir_matriz(np.concatenate((A, b.T), axis = 1))
    x = A_inv[:, 0]

    return x


# Modificación de la función de cambio de base
def cambio_base_matriz(B_prime, B):
    """
    Calcula la matriz de cambio de base de B a B' usando `resolver_sistema`.
    """
    n = len(B)
    C = np.zeros((n, n))
    
    # Resolver los sistemas B_prime_i = C * B_i
    for i in range(n):
        C[:, i] = # Usamos resolver_sistema en lugar de np.linalg.solve
    
    return C



SyntaxError: invalid syntax (3745222319.py, line 31)

In [51]:
A = np.array([[1,  2,  4],
       [2,  1,  2],
       [2,  6,  1]])
b = np.array([[8, 1, 4]])

def resolver_sistema(A, b):
    A_inv = invertir_matriz(np.concatenate((A, b.T), axis = 1))
    x = A_inv[:, 0]

    return x
resolver_sistema(A, b)

array([-2.,  1.,  2.])

In [48]:
def invertir_matriz(A):
    n = A.shape[0]
    I = np.identity(n)
    conc = np.concatenate((A, I), axis = 1)

    M = escalonar_filas(conc)

    A_inv = np.round(back_substitution(M), 2)


    return A_inv
invertir_matriz(A)

array([[-0.33,  0.67,  0.  ],
       [ 0.06, -0.21,  0.18],
       [ 0.3 , -0.06, -0.09]])

In [50]:



A_inv = invertir_matriz(np.concatenate((A, b.T), axis = 1))
A_inv
#x = A_inv[:, -1]
#x
#np.concatenate((A, b.T), axis = 1)


array([[-2.  , -0.33,  0.67,  0.  ],
       [ 1.  ,  0.06, -0.21,  0.18],
       [ 2.  ,  0.3 , -0.06, -0.09]])

### Ayuda. Cambios de Base.

Dadas dos bases $ B = \{\mathbf{v}_1, \mathbf{v}_2, \dots, \mathbf{v}_n\} $ y $ B' = \{\mathbf{w}_1, \mathbf{w}_2, \dots, \mathbf{w}_n\} $, queremos encontrar la **matriz de cambio de base** $ C $, que permite expresar un vector en la base $ B $ como combinación lineal de los vectores de la base $ B' $.

La matriz de cambio de base $ C $ debe satisfacer:
$$
[\mathbf{x}]_B = C [\mathbf{x}]_{B'}
$$
Es decir, el vector $ \mathbf{x} $ en la base $ B' $ se puede transformar a la base $ B $ multiplicando por la matriz $ C $.

Para obtener $ C $, necesitamos expresar cada vector de $ B' $, $ \mathbf{w}_i $, como combinación lineal de los vectores de $ B $, es decir, encontrar los coeficientes $ c_{ij} $ tales que:
$$
\mathbf{w}_i = c_{i1} \mathbf{v}_1 + c_{i2} \mathbf{v}_2 + \dots + c_{in} \mathbf{v}_n
$$
Esto se puede expresar de forma matricial como:
$$
\mathbf{w}_i = C_i \mathbf{v}
$$
Donde:
- $ \mathbf{w}_i $ es el $ i $-ésimo vector de la base $ B' $,
- $ C_i $ es el $ i $-ésimo vector columna de la matriz de cambio de base $ C $,
- $ \mathbf{v} $ es el vector de la base $ B $ con los vectores $ \mathbf{v}_1, \mathbf{v}_2, \dots, \mathbf{v}_n $.

Entonces, para hallar C:

1. Para cada vector $ \mathbf{w}_i \in B' $, resolvemos el sistema $ B \cdot C_i = \mathbf{w}_i $.
2. Los coeficientes $ C_i $ formarán las columnas de la matriz de cambio de base $ C $.
3. Esto lo hacemos utilizando un algoritmo de resolución de sistemas lineales (sin `np.linalg.solve`, usando eliminación gaussiana, por ejemplo).

El bucle del código:
```
for i in range(n):
    C[:, i] = ...
```

Resuelve el sistema para cada vector $ \mathbf{w}_i \in B' $ y almacena los coeficientes en las columnas de la matriz $ C $.

In [56]:
import numpy as np

def elim_gaussiana(A):
    cant_op = 0
    m=A.shape[0]
    n=A.shape[1]
    Ac = A.copy()
    
    if m!=n:
        print('Matriz no cuadrada')
        return
    
    ## desde aqui -- CODIGO A COMPLETAR





                
    ## hasta aqui
            
    L = np.tril(Ac,-1) + np.eye(A.shape[0]) 
    U = np.triu(Ac)
    
    return L, U, cant_op


def main():
    n = 7
    B = np.eye(n) - np.tril(np.ones((n,n)),-1) 
    B[:n,n-1] = 1
    print('Matriz B \n', B)
    
    L,U,cant_oper = elim_gaussiana(B)
    
    print('Matriz L \n', L)
    print('Matriz U \n', U)
    print('Cantidad de operaciones: ', cant_oper)
    print('B=LU? ' , 'Si!' if np.allclose(np.linalg.norm(B - L@U, 1), 0) else 'No!')
    print('Norma infinito de U: ', np.max(np.sum(np.abs(U), axis=1)) )

if __name__ == "__main__":
    main()

Matriz B 
 [[ 1.  0.  0.  0.  0.  0.  1.]
 [-1.  1.  0.  0.  0.  0.  1.]
 [-1. -1.  1.  0.  0.  0.  1.]
 [-1. -1. -1.  1.  0.  0.  1.]
 [-1. -1. -1. -1.  1.  0.  1.]
 [-1. -1. -1. -1. -1.  1.  1.]
 [-1. -1. -1. -1. -1. -1.  1.]]
Matriz L 
 [[ 1.  0.  0.  0.  0.  0.  0.]
 [-1.  1.  0.  0.  0.  0.  0.]
 [-1. -1.  1.  0.  0.  0.  0.]
 [-1. -1. -1.  1.  0.  0.  0.]
 [-1. -1. -1. -1.  1.  0.  0.]
 [-1. -1. -1. -1. -1.  1.  0.]
 [-1. -1. -1. -1. -1. -1.  1.]]
Matriz U 
 [[1. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 1.]]
Cantidad de operaciones:  0
B=LU?  No!
Norma infinito de U:  2.0


In [58]:
A


array([[1, 2, 4],
       [2, 1, 2],
       [2, 6, 1]])

In [60]:
Ac = A.copy()
L = np.tril(Ac,-1) + np.eye(A.shape[0]) 
L

array([[1., 0., 0.],
       [2., 1., 0.],
       [2., 6., 1.]])

In [61]:
U = np.triu(Ac)
U

array([[1, 2, 4],
       [0, 1, 2],
       [0, 0, 1]])

In [62]:
def resolver_L(L, b):
    res = resolver_sistema(L, b)
    return res

resolver_L(L, b)
    

array([  8., -15.,  78.])

In [65]:
def resolver_U(U, b):
    res = resolver_sistema(U, b)
    return res
    
resolver_U(U, b)

array([ 6., -7.,  4.])